In [2]:
# 导入必要的库
import time    # 导入时间模块，用于控制等待时间
import pandas as pd    # 用于数据存储与导出
import os    # 导入操作系统模块，用于文件路径操作
import traceback

# 从selenium库导入需要的功能
from selenium import webdriver    # 导入主浏览器控制模块
from selenium.webdriver.common.by import By    # 导入元素定位方式
from selenium.webdriver.support.ui import WebDriverWait    # 导入等待功能
from selenium.webdriver.support import expected_conditions as EC    # 导入等待条件
from selenium.common.exceptions import (
    NoSuchElementException, TimeoutException, StaleElementReferenceException
)

# 导入浏览器服务配置
from selenium.webdriver.edge.service import Service

In [4]:
# 配置参数
TARGET_URL = "https://zu.fang.com"    # 目标网址
MAX_PAGES = 20    # 最大爬取页数
AREA_FILTERS = [    # 区域筛选步骤（按实际网页调整XPath）
    "//dl[@id='rentid_D04_01']//a[text()='海淀']",    # 第一步：点击“海淀”
    "//div[@id='rentid_D04_08']//a[text()='上地']"     # 第二步：点击“上地”
]
HOUSE_LIST_XPATH = "//dd[contains(@class, 'info rel')]"    # 房源列表容器XPath
NEXT_PAGE_XPATH = "//a[contains(text(), '下一页')]"    # 下一页按钮XPath

In [6]:
def setup_optimized_driver():
    # 初始化带优化配置的WebDriver
    service = Service(r"D:\APP\WebDrivers\msedgedriver.exe")
    options = webdriver.EdgeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--blink-settings=imagesEnabled=false')    # 禁用图片加载
    options.add_argument('--disable-gpu')    # 禁用GPU加速
    options.add_argument('--incognito')
    
    driver = webdriver.Edge(service=service, options=options)
    return driver

In [10]:
def apply_area_filters(driver):
    # 应用区域筛选条件（保证爬取范围正确）
    driver.get(TARGET_URL)
    for filter_xpath in AREA_FILTERS:
        try:
            # 等待筛选按钮可点击后再点击
            filter_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, filter_xpath))
            )
            filter_btn.click()
            # 随机等待1-2秒（模拟人工操作，避免反爬）
            time.sleep(random.uniform(1, 2))
        except Exception as e:
            print(f"筛选区域失败：{e}")
            raise    # 筛选失败直接终止，避免爬错区域

In [12]:
def click_next_page(driver, current_page):
    """
    点击下一页按钮，封装翻页逻辑
    参数：
        driver: 浏览器驱动实例
        current_page: 当前页码（用于日志输出）
    返回：
        tuple: (是否成功翻页, 新页码)
    """
    max_retry = 3    # 最大重试次数
    for retry in range(max_retry):
        try:
            # 等待下一页按钮可点击
            next_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, NEXT_PAGE_XPATH))
            )
            next_btn.click()
            # 随机等待2-3秒，确保页面加载完成
            time.sleep(random.uniform(2, 3))
            return (True, current_page + 1)    # 成功翻页，返回新页码
        
        except (TimeoutException, NoSuchElementException):
            # 重试耗尽则判定为无下一页
            if retry == max_retry - 1:
                print(f"第{current_page}页：无下一页按钮或按钮不可点击")
                return (False, current_page)
            else:
                print(f"第{current_page}页：第{retry+1}次点击下一页失败，重试...")
                time.sleep(2)    # 重试间隔
        
        except Exception as e:
            # 其他异常（如点击后页面无响应）
            if retry == max_retry - 1:
                print(f"第{current_page}页：翻页失败，未知错误：{str(e)[:50]}")
                return (False, current_page)
            else:
                print(f"第{current_page}页：第{retry+1}次翻页异常，重试...")
                time.sleep(2)

In [14]:
def extract_current_page(driver, page_num):
    # 提取当前页面的租房信息（基于您提供的代码修改）
    current_page_data = []    # 存储当前页数据
    try:
        # 等待房源列表加载（使用您提供的XPath）
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'list-box mt20')]//div[contains(@class, 'houseList')]"))
        )
        
        # 获取当前页所有房源元素（使用您提供的XPath）
        house_items = driver.find_elements(By.XPATH, "//div[contains(@class, 'list-box mt20')]//dd[contains(@class, 'info rel')]")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, ".//dd[contains(@class, 'info rel')]//p[@class='title']/a"))
        )
        
        print(f"第{page_num}页共找到 {len(house_items)} 条房源")
        
        for index, item in enumerate(house_items, 1):
            try:
                # 提取标题
                title = item.find_element(By.XPATH, ".//p[@class='title']/a").text
                
                # 提取“整租 | 户型 | 面积 | 朝向”（文本拆分法）
                info_p = item.find_element(By.XPATH, ".//p[@class='font15 mt12 bold']")
                info_text = info_p.text    # 示例："整租 | 4室3厅 | 314㎡ | 朝南北"
                info_parts = info_text.split("|")    # 按竖线分隔
            
                rent_mode = info_parts[0].strip() if len(info_parts) > 0 else ""    # 整租类型
                house_type = info_parts[1].strip() if len(info_parts) > 1 else ""    # 户型
                area = info_parts[2].strip() if len(info_parts) > 2 else ""    # 面积
                direction = info_parts[3].strip() if len(info_parts) > 3 else ""    # 朝向
               
                # 提取价格
                price = item.find_element(By.XPATH, ".//span[@class='price']").text
                # 提取位置
                location = item.find_element(By.XPATH, ".//p[@class='gray6 mt12']").text
                # 提取地铁距离
                subway_info = ""
                try:
                    subway_info = item.find_element(By.XPATH, ".//span[@class='note subInfor']").text
                except NoSuchElementException:
                    subway_info = ""
                
                # 提取标签
                tag_elements = item.find_elements(By.XPATH, ".//span[contains(@class, 'note')]//a")
                tags = [tag.text for tag in tag_elements]
                tags_str = ", ".join(tags) if tags else ""
                
                # 构造房源数据字典
                current_house = {
                    "页码": page_num,    # 增加页码标识
                    "标题": title,
                    "租房类型": rent_mode,
                    "户型": house_type,
                    "面积": area,
                    "朝向": direction,
                    "价格": price,
                    "位置": location,
                    "地铁距离": subway_info,
                    "标签": tags_str
                }
                current_page_data.append(current_house)
                print(f"第{page_num}页：已提取 {index}/{len(house_items)} 条")
                
            except StaleElementReferenceException:
                print(f"第{page_num}页：第{index}条房源元素已失效，跳过")
                continue
            except Exception as e:
                print(f"第{page_num}页：第{index}条提取失败: {str(e)[:50]}")
        
        return current_page_data
    
    except TimeoutException:
        print(f"第{page_num}页：等待房源加载超时")
        return []
    except Exception as e:
        print(f"第{page_num}页：整体提取失败: {e}")
        return []

In [16]:
def crawl_all_pages():
    # 一次性爬取所有目标页数
    driver = setup_optimized_driver()
    all_data = []
    current_page = 1
    
    try:
        # 应用区域筛选
        apply_area_filters(driver)
        
        while current_page <= MAX_PAGES:
            # 提取当前页数据
            page_data = extract_current_page(driver, current_page)
            if page_data:
                all_data.extend(page_data)
                print(f"第{current_page}页提取完成，累计 {len(all_data)} 条数据")
            
            # 达到最大页数则停止
            if current_page == MAX_PAGES:
                print(f"已完成{MAX_PAGES}页爬取")
                break
            
            # 每5页清理一次缓存
            if current_page % 5 == 0:
                driver.delete_all_cookies()
                print(f"已清理缓存，释放内存")
            
            # 调用封装的下一页函数
            has_next, new_page = click_next_page(driver, current_page)
            if has_next:
                current_page = new_page    # 更新页码
            else:
                break    # 无下一页，终止爬取
        
    except Exception as e:
        print(f"爬取中断：{e}")
        print(traceback.format_exc())
    finally:
        driver.quit()
        print("浏览器已关闭")
    
    # 保存数据
    if all_data:
        pd.DataFrame(all_data).to_csv("all_rent_data.csv", index=False, encoding="utf-8-sig")
        print(f"共爬取{len(all_data)}条数据，已保存至all_rent_data.csv")
    else:
        print("未爬取到任何数据")

In [38]:
if __name__ == "__main__":
    crawl_all_pages()

第1页共找到 60 条房源
第1页：已提取 1/60 条
第1页：已提取 2/60 条
第1页：已提取 3/60 条
第1页：已提取 4/60 条
第1页：已提取 5/60 条
第1页：已提取 6/60 条
第1页：已提取 7/60 条
第1页：已提取 8/60 条
第1页：已提取 9/60 条
第1页：已提取 10/60 条
第1页：已提取 11/60 条
第1页：已提取 12/60 条
第1页：已提取 13/60 条
第1页：已提取 14/60 条
第1页：已提取 15/60 条
第1页：已提取 16/60 条
第1页：已提取 17/60 条
第1页：已提取 18/60 条
第1页：已提取 19/60 条
第1页：已提取 20/60 条
第1页：已提取 21/60 条
第1页：已提取 22/60 条
第1页：已提取 23/60 条
第1页：已提取 24/60 条
第1页：已提取 25/60 条
第1页：已提取 26/60 条
第1页：已提取 27/60 条
第1页：已提取 28/60 条
第1页：已提取 29/60 条
第1页：已提取 30/60 条
第1页：已提取 31/60 条
第1页：已提取 32/60 条
第1页：已提取 33/60 条
第1页：已提取 34/60 条
第1页：已提取 35/60 条
第1页：已提取 36/60 条
第1页：已提取 37/60 条
第1页：已提取 38/60 条
第1页：已提取 39/60 条
第1页：已提取 40/60 条
第1页：已提取 41/60 条
第1页：已提取 42/60 条
第1页：已提取 43/60 条
第1页：已提取 44/60 条
第1页：已提取 45/60 条
第1页：已提取 46/60 条
第1页：已提取 47/60 条
第1页：已提取 48/60 条
第1页：已提取 49/60 条
第1页：已提取 50/60 条
第1页：已提取 51/60 条
第1页：已提取 52/60 条
第1页：已提取 53/60 条
第1页：已提取 54/60 条
第1页：已提取 55/60 条
第1页：已提取 56/60 条
第1页：已提取 57/60 条
第1页：已提取 58/60 条
第1页：已提取 59/60 条
第1页：已提取 60/60 条
第1页提取完成，累计 60 条数据
第2页共找到 60 条房源
第2页